Notebook to train the digit classifier

In [34]:
import itertools

from keras.callbacks import ReduceLROnPlateau
from keras.layers import Conv2D
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import MaxPool2D
from keras.models import Sequential
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import tensorflow as tf

Load the data and apply basic preprocessing

In [39]:
df = pd.read_csv("dataset/char74k/char74k.csv")

X = df.drop(labels=["label"], axis=1)
X /= 255.0
X = X.values.reshape(-1, 28, 28, 1)
y = to_categorical(df["label"], num_classes=10)

Split the data into train and validation sets

In [41]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

Initialize the model and optimizer

In [43]:
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(5, 5), padding="Same", activation="relu", input_shape=(28, 28, 1)))
model.add(Conv2D(filters=32, kernel_size=(5, 5), padding="Same", activation="relu"))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(filters=64, kernel_size=(3, 3), padding="Same", activation="relu"))
model.add(Conv2D(filters=64, kernel_size=(3, 3), padding="Same", activation="relu"))
model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(256, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation="softmax"))
optimizer = RMSprop(learning_rate=0.001, rho=0.9, epsilon=1e-08)
model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])

Initialize the data augmentor

In [44]:
image_data_generator = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=False,
    vertical_flip=False
)
image_data_generator.fit(X_train)

Train the model

In [ ]:
history = model.fit(
    image_data_generator.flow(X_train, y_train, batch_size=86),
    epochs=30,
    verbose=2,
    callbacks=[ReduceLROnPlateau(
        monitor='val_accuracy',
        factor=0.5,
        patience=3,
        verbose=1,
        min_lr=0.00001
    )],
    validation_data=(X_val, y_val),
    steps_per_epoch=X_train.shape[0] // 86
)

Print the train/validation loss/accuracy curves

In [ ]:
fig, ax = plt.subplots(2, 1)

ax[0].plot(history.history["loss"], color="b", label="Train Loss")
ax[0].plot(history.history["val_loss"], color="r", label="Validation Loss")
ax[0].set_xlabel("Epoch")
ax[0].legend(loc="best")

ax[1].plot(history.history["accuracy"], color="b", label="Train Accuracy")
ax[1].plot(history.history["val_accuracy"], color="r",label="Validation Accuracy")
ax[1].set_xlabel("Epoch")
ax[1].legend(loc="best")

Print the confusion matrix

In [ ]:
def plot_confusion_matrix(
    cm, classes, normalize=False, title="Confusion Matrix", cmap=plt.cm.Blues
):
    plt.imshow(cm, interpolation="nearest", cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype("float") / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(
            j,
            i,
            cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black"
        )

    plt.tight_layout()
    plt.ylabel("True Label")
    plt.xlabel("Predicted Label")

y_pred = model.predict(X_val)
y_pred_classes = np.argmax(y_pred, axis=1) 
y_true = np.argmax(y_val, axis=1) 
confusion_matrix = confusion_matrix(y_true, y_pred_classes)
plot_confusion_matrix(confusion_matrix, classes=range(10)) 

Save the model

In [ ]:
tf.keras.saving.save_model(
    model, "./model/char74k.h5", overwrite=True, save_format="h5"
)